In [30]:
import os
import base64
from googleapiclient.discovery import build
import requests
import openai
import re
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [37]:
# Define the scopes required to access the Gmail API
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate():
    """Authenticate with the Gmail API."""
    creds = None
    # Check if there are existing credentials
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')
    # If there are no (valid) credentials, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

In [47]:
def email_checker(body):
    api_url = "https://api.openai.com/v1/chat/completions"
        
        # Your API key
    openai.api_key = ""

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0125",
        messages=[
                {"role": "system", "content": """You will read an email that could potentially be from a recruiter. 
                You should read the email and respond ONLY by one of the three options:
                    1- "Accepted" : in the case that the email is an acceptance letter or asking for an interview.
                    2- "Rejected" : in the case that the email is a rejection letter.
                    3- "Not a job email" : in the case that the email is not related to jobs or when the email is not an admission decision. \n\n 
                The email starts after this."""},
                {"role": "user", "content": body},
                ]
            )

  
    return response['choices'][0]['message']['content']


In [48]:
def fetch_email_body(message_id):
    """Fetch the body of a specific email message."""
    creds = authenticate()
    service = build('gmail', 'v1', credentials=creds)
    msg = service.users().messages().get(userId='me', id=message_id).execute()
    payload = msg['payload']
    parts = payload.get('parts', [])
    for part in parts:
        if part['body'] and 'data' in part['body']:
            body = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
            # Clean up the body by removing leading and trailing whitespace
            body = body.strip()
            # Remove extra whitespace within the body
            body = re.sub(r'\s+', ' ', body)
            return body
    return ""

def fetch_unread_emails():
    """Fetch unread emails from Gmail."""
    creds = authenticate()
    service = build('gmail', 'v1', credentials=creds)
    # Call the Gmail API to fetch unread messages
    results = service.users().messages().list(userId='me', q='is:unread').execute()
    messages = results.get('messages', [])
    if not messages:
        print('No unread messages found.')
    else:
        print('Unread Messages:')
        for message in messages:
            msg_id = message['id']
            body = fetch_email_body(msg_id)
            # Check if the body is empty
            if body.strip():
                print(email_checker(body))

In [51]:
fetch_unread_emails()

Unread Messages:
Rejected
